<a href="https://colab.research.google.com/github/Srushanth/Binary-Classification-with-a-Software-Defects-Dataset/blob/main/development%5Cnotebooks%5COptuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install optuna

In [6]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn import ensemble
from sklearn import model_selection

import optuna
from functools import partial

In [18]:
def optimize(trial, X, y):
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    n_estimators = trial.suggest_int("n_estimators", 100, 1500)
    max_depth = trial.suggest_int("max_depth", 3, 15)
    max_features = trial.suggest_int("max_features", 1, 20)

    model = ensemble.RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        criterion=criterion
    )

    kf = model_selection.StratifiedKFold(n_splits=5)
    accuracies = []
    for idx in kf.split(X=X, y=y):
        train_idx, test_idx = idx[0], idx[1]
        xtrain = X[train_idx]
        ytrain = y[train_idx]

        xtest = X[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        preds = model.predict(xtest)
        fold_acc = metrics.accuracy_score(ytest, preds)
        accuracies.append(fold_acc)

    return -1.0 * np.mean(accuracies)

In [19]:
  df = pd.read_csv("./sample_data/train.csv")
  X = df.drop(columns=["defects"]).values
  y = df["defects"].values

In [20]:
optimization_function = partial(optimize, X=X, y=y)

In [21]:
study = optuna.create_study(direction="minimize")
study.optimize(optimization_function, n_trials=15)

[W 2023-10-12 13:24:23,582] Trial 0 failed with parameters: {'criterion': 'entropy', 'n_estimators': 747, 'max_depth': 7, 'max_features': 16} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-18-32237de8f769>", line 24, in optimize
    model.fit(xtrain, ytrain)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 473, in fit
    trees = Parallel(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py", line 63, in __call__
    return super().__call__(iterable_with_config)
  File "/usr/local/lib/python3.10/dist-packages/joblib/parallel.py", line 1863, in __call__
    return output if self.return_generator else list(output)
  File "/usr/local/lib/python3.10/dist-packages/joblib/parallel.py", line 1792, in _get_sequential_output
    res = func(*args

KeyboardInterrupt: ignored